## Node-level caching in LangGraph



In [1]:
from langgraph.cache.memory import InMemoryCache
from langgraph.graph import StateGraph
from langgraph.types import CachePolicy
from typing_extensions import TypedDict
import time

In [2]:
class State(TypedDict):
    x: int
    result: int

In [29]:
builder = StateGraph(State)

In [30]:
def expensive_node(state: State) -> dict[str, int]:
    time.sleep(2)
    return {"result": state["x"] * 2}

In [31]:
builder.add_node(
    "expensive_node", 
    expensive_node, 
    cache_policy=CachePolicy(ttl=120)
)

In [32]:
builder.set_entry_point("expensive_node")
builder.set_finish_point("expensive_node")

In [33]:
cache = InMemoryCache()


In [34]:
graph = builder.compile(cache=cache)

In [45]:
print(graph.invoke({"x": 6}, stream_mode="updates"))

[{'expensive_node': {'result': 12}, '__metadata__': {'cached': True}}]


In [43]:
cache.clear()

## Two expensive nodes

In [46]:
class State(TypedDict):
    x: int
    y: int
    result: int

In [47]:
builder = StateGraph(State)

In [49]:
def expensive_node(state: State) -> dict[str, int]:
    time.sleep(2)
    return {"result": state["x"] * 2}

In [50]:
def expensive_node2(state: State) -> dict[str, int]:
    time.sleep(2)
    return {"result": state["y"] * 20}

In [51]:
builder.add_node(
    "expensive_node",
    expensive_node,
    cache_policy=CachePolicy(
        ttl=120
    )
)

In [52]:
builder.add_node(
    "expensive_node2",
    expensive_node2,
    cache_policy=CachePolicy(
        ttl=30
    )
)

In [53]:
builder.add_edge("expensive_node", "expensive_node2")
builder.set_entry_point("expensive_node")

In [54]:
cache = InMemoryCache()
graph = builder.compile(cache=cache)

In [68]:
print(graph.invoke({"x": 6,"y": 8}, stream_mode="updates"))

[{'expensive_node': {'result': 12}, '__metadata__': {'cached': True}}, {'expensive_node2': {'result': 160}}]


In [59]:
cache.clear()

In [62]:
cache._cache

{('__pregel_ns_writes',
  '__main__.expensive_node',
  'expensive_node'): {'f4c56c93b6a9632f57c7ff6c9e03157e': ('msgpack',
   b'\xc79\x00\x93\xabcollections\xa5deque\x92\x92\xa6result\x0c\x92\xb9branch:to:expensive_node2\xc0',
   1750051821.612292)},
 ('__pregel_ns_writes',
  '__main__.expensive_node2',
  'expensive_node2'): {'8f152a5b6c4a173c478b25f2cc993866': ('msgpack',
   b'\xc7\x1e\x00\x93\xabcollections\xa5deque\x91\x92\xa6result\xcc\xa0',
   1750051733.616265)}}

In [66]:
cache.clear(namespaces={
    ('__pregel_ns_writes',
  '__main__.expensive_node',
  'expensive_node')
}
)

In [69]:
cache.clear(namespaces={
    ('__pregel_ns_writes',
  '__main__.expensive_node2',
  'expensive_node2')
})

In [70]:
print(graph.invoke({"x": 6,"y": 8}, stream_mode="updates"))

[{'expensive_node': {'result': 12}, '__metadata__': {'cached': True}}, {'expensive_node2': {'result': 160}}]
